In [55]:
#!pip install pymysql

In [56]:
import pandas as pd
import numpy as np
import pymysql as mysql
import mysql.connector 
from mysql.connector import errorcode

In [98]:
#Import the data and check for null value
df = pd.read_csv('train.csv')
df["Order Date"] = pd.to_datetime(df["Order Date"],format='%Y-%m-%d')
df["Ship Date"] = pd.to_datetime(df["Ship Date"],format='%Y-%m-%d')
df['Sales'] = df['Sales'].round(2)
df['Profit'] = df['Profit'].round(2)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Row ID         9994 non-null   int64         
 1   Order ID       9994 non-null   object        
 2   Order Date     9994 non-null   datetime64[ns]
 3   Ship Date      9994 non-null   datetime64[ns]
 4   Ship Mode      9994 non-null   object        
 5   Customer ID    9994 non-null   object        
 6   Customer Name  9994 non-null   object        
 7   Segment        9994 non-null   object        
 8   Country        9994 non-null   object        
 9   City           9994 non-null   object        
 10  State          9994 non-null   object        
 11  Postal Code    9994 non-null   int64         
 12  Region         9994 non-null   object        
 13  Product ID     9994 non-null   object        
 14  Category       9994 non-null   object        
 15  Sub-Category   9994 n

In [58]:
# Check the potential key for data model
key_check = ['Row ID', 'Order ID', 'Customer ID', 'Postal Code', 'Product ID']

for x in key_check:
    print(x + ': ' + str(len(df[x].unique())))

Row ID: 9994
Order ID: 5009
Customer ID: 793
Postal Code: 631
Product ID: 1862


In [59]:
# Extract the transaction table and add key product_id
transaction = df[['Row ID', 'Order ID','Product ID','Sales', 'Product Name','Quantity','Discount','Profit']].copy()
dict ={'Row ID': 'transaction_id',
      'Order ID': 'order_id',
      'Product ID': 'product_code',
      'Sales': 'sales',
      'Product Name': 'product_name',
      'Quantity': 'quantity',
      'Discount': 'discount',
      'Profit': 'profit'}
transaction.rename(columns=dict, inplace=True)
transaction['product_id'] = pd.factorize(transaction['product_code'] + '_' + transaction['product_name'])[0]
transaction = transaction.drop(columns=['product_code','product_name'])
transaction

,transaction_id,order_id,sales,quantity,discount,profit,product_id
0,1,CA-2016-152156,261.96,2,0.00,41.91,0
1,2,CA-2016-152156,731.94,3,0.00,219.58,1
2,3,CA-2016-138688,14.62,2,0.00,6.87,2
3,4,US-2015-108966,957.58,5,0.45,-383.03,3
4,5,US-2015-108966,22.37,2,0.20,2.52,4
...,...,...,...,...,...,...,...
9989,9990,CA-2014-110422,25.25,3,0.20,4.10,1182
9990,9991,CA-2017-121258,91.96,2,0.00,15.63,1793
9991,9992,CA-2017-121258,258.58,2,0.20,19.39,298
9992,9993,CA-2017-121258,29.60,4,0.00,13.32,964


In [60]:
# Extract the order table and add key address_id
order = df[['Order ID','Order Date','Ship Date','Ship Mode','Customer ID','Customer Name','Segment','Postal Code','Country','Region','City','State']].copy()
dict = {'Order ID':'order_id','Order Date':'order_date','Ship Date':'ship_date','Ship Mode':'ship_mode','Customer ID':'customer_id','Customer Name':'customer_name','Segment':'segment','Postal Code':'postal_code','Country':'country','City':'city','State':'state','Region':'region'}
order.rename(columns=dict, inplace = True)
order['address_id'] = pd.factorize(order['postal_code'].astype(str) + '_' + order['country'] + '_' + order['region'] + '_' + order['city'] + '_' + order['state'])[0]
order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   order_id       9994 non-null   object        
 1   order_date     9994 non-null   datetime64[ns]
 2   ship_date      9994 non-null   datetime64[ns]
 3   ship_mode      9994 non-null   object        
 4   customer_id    9994 non-null   object        
 5   customer_name  9994 non-null   object        
 6   segment        9994 non-null   object        
 7   postal_code    9994 non-null   int64         
 8   country        9994 non-null   object        
 9   region         9994 non-null   object        
 10  city           9994 non-null   object        
 11  state          9994 non-null   object        
 12  address_id     9994 non-null   int64         
dtypes: datetime64[ns](2), int64(2), object(9)
memory usage: 1015.1+ KB


In [61]:
# Extract the Order_detailed table and remove duplicates
order_detailed = order[['order_id','order_date','ship_date','ship_mode','customer_id','address_id']]
order_detailed = order_detailed.drop_duplicates(subset=['order_id','order_date','ship_date','ship_mode'])
order_detailed

,order_id,order_date,ship_date,ship_mode,customer_id,address_id
0,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,0
2,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,1
3,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,2
5,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,BH-11710,3
12,CA-2017-114412,2017-04-15,2017-04-20,Standard Class,AA-10480,4
...,...,...,...,...,...,...
9986,CA-2016-125794,2016-09-29,2016-10-03,Standard Class,ML-17410,141
9987,CA-2017-163629,2017-11-17,2017-11-21,Standard Class,RA-19885,506
9989,CA-2014-110422,2014-01-21,2014-01-23,Second Class,TB-21400,123
9990,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,146


In [62]:
# Extract the customer Information table and remove duplicates
cus_info = order[['customer_id','customer_name','segment']]
cus_info = cus_info.drop_duplicates(subset=['customer_id'])
cus_info

,customer_id,customer_name,segment
0,CG-12520,Claire Gute,Consumer
2,DV-13045,Darrin Van Huff,Corporate
3,SO-20335,Sean O'Donnell,Consumer
5,BH-11710,Brosina Hoffman,Consumer
12,AA-10480,Andrew Allen,Consumer
...,...,...,...
8666,CJ-11875,Carl Jackson,Corporate
9209,RS-19870,Roy Skaria,Home Office
9399,SC-20845,Sung Chung,Consumer
9441,RE-19405,Ricardo Emerson,Consumer


In [63]:
# Extract the customer address table and remove duplicates
cus_ad= order[['address_id', 'postal_code','country','region','city','state']]
cus_ad = cus_ad.drop_duplicates(subset='address_id')
cus_ad['postal_code']=cus_ad['postal_code'].fillna(1)
cus_ad

,address_id,postal_code,country,region,city,state
0,0,42420,United States,South,Henderson,Kentucky
2,1,90036,United States,West,Los Angeles,California
3,2,33311,United States,South,Fort Lauderdale,Florida
5,3,90032,United States,West,Los Angeles,California
12,4,28027,United States,South,Concord,North Carolina
...,...,...,...,...,...,...
9760,627,72762,United States,South,Springdale,Arkansas
9801,628,95240,United States,West,Lodi,California
9834,629,77571,United States,Central,La Porte,Texas
9868,630,45040,United States,East,Mason,Ohio


In [64]:
# Extract the sub categories table and remove duplicates
sub_cat = df[['Category','Sub-Category']].copy()
sub_cat=sub_cat.rename(columns={'Category':'categories','Sub-Category':'sub_categories'})
sub_cat=sub_cat.drop_duplicates(subset=['sub_categories'])
sub_cat

,categories,sub_categories
0,Furniture,Bookcases
1,Furniture,Chairs
2,Office Supplies,Labels
3,Furniture,Tables
4,Office Supplies,Storage
5,Furniture,Furnishings
6,Office Supplies,Art
7,Technology,Phones
8,Office Supplies,Binders
9,Office Supplies,Appliances


In [65]:
# Extract the product table, add key product_id and remove duplicates
product = df[['Product ID','Sub-Category','Product Name']].copy()
dict = {'Product ID':'product_code','Category':'category','Sub-Category':'sub_categories','Product Name':'product_name'}
product.rename(columns=dict, inplace = True)
product['product_id'] = pd.factorize(product['product_code'] + '_' + product['product_name'])[0]
product=product.drop_duplicates(subset=['product_id'])
product

,product_code,sub_categories,product_name,product_id
0,FUR-BO-10001798,Bookcases,Bush Somerset Collection Bookcase,0
1,FUR-CH-10000454,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",1
2,OFF-LA-10000240,Labels,Self-Adhesive Address Labels for Typewriters b...,2
3,FUR-TA-10000577,Tables,Bretford CR4500 Series Slim Rectangular Table,3
4,OFF-ST-10000760,Storage,Eldon Fold 'N Roll Cart System,4
...,...,...,...,...
9521,TEC-PH-10002817,Phones,RCA ViSYS 25425RE1 Corded phone,1889
9562,TEC-MA-10003589,Machines,Cisco 8961 IP Phone Charcoal,1890
9604,OFF-AP-10003099,Appliances,"Eureka Hand Vacuum, Bagless",1891
9673,TEC-PH-10002645,Phones,LG G2,1892


In [66]:
# Extract the customer table and remove duplicates
cus_iden = cus_info[['customer_id','customer_name','segment']].copy()
cus_iden = cus_iden.drop_duplicates(subset='customer_id')
cus_iden

,customer_id,customer_name,segment
0,CG-12520,Claire Gute,Consumer
2,DV-13045,Darrin Van Huff,Corporate
3,SO-20335,Sean O'Donnell,Consumer
5,BH-11710,Brosina Hoffman,Consumer
12,AA-10480,Andrew Allen,Consumer
...,...,...,...
8666,CJ-11875,Carl Jackson,Corporate
9209,RS-19870,Roy Skaria,Home Office
9399,SC-20845,Sung Chung,Consumer
9441,RE-19405,Ricardo Emerson,Consumer


In [85]:
# Connecting to the local MySQL database
cnx = mysql.connector.connect(user='root', password='Mayvitinh1995$',
                              database='superstore')
cursor = cnx.cursor(buffered=True)

In [86]:
# Prepare the query scripts for create table in the database
TABLES = {}
TABLES['product'] = (
    "CREATE TABLE `product` ("
    "  `product_code` varchar(65) NOT NULL,"
    "  `sub_categories` varchar(45) NOT NULL,"
    "  `product_name` varchar(1000) NOT NULL,"
    "  `product_id` int NOT NULL,"
    "  PRIMARY KEY (`product_id`)"
    ") ENGINE=InnoDB")

TABLES['transaction'] = (
    "CREATE TABLE `transaction` ("
    "  `transaction_id` int NOT NULL,"
    "  `order_id` varchar(45) NOT NULL,"
    "  `product_id` int NOT NULL,"
    "  `sales` numeric(10,2) NOT NULL,"
    "  `quantity` int NOT NULL,"
    "  `discount` numeric(10,2) NOT NULL,"
    "  `profit` numeric(10,2) NOT NULL,"
    "  PRIMARY KEY (`transaction_id`)"
    ") ENGINE=InnoDB")

TABLES['order_detailed'] = (
    "CREATE TABLE `order_detailed` ("
    "  `order_id` varchar(65) NOT NULL,"
    "  `order_date` date NOT NULL,"
    "  `ship_date` date NOT NULL,"
    "  `ship_mode` varchar(45) NOT NULL,"
    "  `customer_id` varchar(45) NOT NULL,"
    "  `address_id` int NOT NULL,"
    "  PRIMARY KEY (`order_id`)"
    ") ENGINE=InnoDB")

TABLES['customer_address'] = (
    "CREATE TABLE `customer_address` ("
    "  `address_id` int NOT NULL,"
    "  `city` varchar(45) NOT NULL,"
    "  `state` varchar(45) NOT NULL,"
    "  `region` varchar(45) NOT NULL,"
    "  `postal_code` int NOT NULL,"
    "  `country` varchar(45) NOT NULL,"
    "  PRIMARY KEY (`address_id`)"
    ") ENGINE=InnoDB")

TABLES['customer_information'] = (
    "CREATE TABLE `customer_information` ("
    "  `customer_id` varchar(65) NOT NULL,"
    "  `customer_name` varchar(45) NOT NULL,"
    "  `segment` varchar(45) NOT NULL,"
    "  PRIMARY KEY (`customer_id`)"
    ") ENGINE=InnoDB")


TABLES['product_detailed'] = (
    "CREATE TABLE `product_detailed` ("
    "  `sub_categories` varchar(65) NOT NULL,"
    "  `categories` varchar(45) NOT NULL,"
    "  PRIMARY KEY (`sub_categories`)"
    ") ENGINE=InnoDB")

In [87]:
# Run the SQL scripts to drop all potential tables if exist
table_names = ['transaction', 'product', 'order_detailed', 'customer_address', 'customer_information', 'product_detailed']
for i in table_names:
    cursor.execute("DROP TABLE IF EXISTS " + i)

In [88]:
# Check the tables in database
cursor.execute("show tables")
for x in cursor:
  print(x) 

In [89]:
#Cretae the tables in databse
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

Creating table product: OK
Creating table transaction: OK
Creating table order_detailed: OK
Creating table customer_address: OK
Creating table customer_information: OK
Creating table product_detailed: OK


In [90]:
# Load the data into tables in datbase
dfs = [transaction, product, order_detailed, cus_ad, cus_iden, sub_cat]

for df, table_name in zip(dfs, table_names):
    cols = ",".join([str(i) for i in df.columns.tolist()])
    for i, row in df.iterrows():
        sql = "INSERT INTO " + table_name + " (" + cols + ") VALUES (" + "%s," * (len(row) - 1) + "%s)"
        cursor.execute(sql, tuple(row))
        cnx.commit()

In [91]:
# Check if loading successfully
cursor.execute("SELECT * FROM walmart.transaction")
for x in cursor:
    print(x)

(1, 'CA-2016-152156', 0, Decimal('261.96'), 2, Decimal('0.00'), Decimal('41.91'))
(2, 'CA-2016-152156', 1, Decimal('731.94'), 3, Decimal('0.00'), Decimal('219.58'))
(3, 'CA-2016-138688', 2, Decimal('14.62'), 2, Decimal('0.00'), Decimal('6.87'))
(4, 'US-2015-108966', 3, Decimal('957.58'), 5, Decimal('0.45'), Decimal('-383.03'))
(5, 'US-2015-108966', 4, Decimal('22.37'), 2, Decimal('0.20'), Decimal('2.52'))
(6, 'CA-2014-115812', 5, Decimal('48.86'), 7, Decimal('0.00'), Decimal('14.17'))
(7, 'CA-2014-115812', 6, Decimal('7.28'), 4, Decimal('0.00'), Decimal('1.97'))
(8, 'CA-2014-115812', 7, Decimal('907.15'), 6, Decimal('0.20'), Decimal('90.72'))
(9, 'CA-2014-115812', 8, Decimal('18.50'), 3, Decimal('0.20'), Decimal('5.78'))
(10, 'CA-2014-115812', 9, Decimal('114.90'), 5, Decimal('0.00'), Decimal('34.47'))
(11, 'CA-2014-115812', 10, Decimal('1706.18'), 9, Decimal('0.20'), Decimal('85.31'))
(12, 'CA-2014-115812', 11, Decimal('911.42'), 4, Decimal('0.20'), Decimal('68.36'))
(13, 'CA-2017-114

In [92]:
# Add foreign key to the tables
cursor.execute("ALTER TABLE transaction ADD FOREIGN KEY (product_id) REFERENCES product(product_id)");
cursor.execute("ALTER TABLE transaction ADD FOREIGN KEY (order_id) REFERENCES order_detailed(order_id)");
cursor.execute("ALTER TABLE order_detailed ADD FOREIGN KEY (address_id) REFERENCES customer_address(address_id)");
cursor.execute("ALTER TABLE order_detailed ADD FOREIGN KEY (customer_id) REFERENCES customer_information(customer_id)");
cursor.execute("ALTER TABLE product ADD FOREIGN KEY (sub_categories) REFERENCES product_detailed(sub_categories)");

In [93]:
# Run the SQL scripts to get the original data format
cursor.execute("""
    select t.transaction_id as 'Row ID', t.order_id as 'Order ID', o.order_date as 'Order Date', o.ship_date as 'Ship Date', o.ship_mode as ' Ship Mode', 
    o.customer_id as 'Customer ID', inf.customer_name as ' Customer Name', inf.segment as 'Segment', ad.country as 'Country', ad.city as 'City', 
    ad.state as 'State', ad.postal_code as 'Postal Code', ad.region as ' Region', p.product_code as 'Product ID', de.categories as 'Category', p.sub_categories as 'Sub-Category',  
    p.product_name as 'Product Name', t.sales as 'Sales', t.Quantity AS 'Quantity', t.discount AS 'Discount', t.profit AS 'Profit'
    from walmart.transaction t
    left join walmart.order_detailed o
    on t.order_id = o.order_id
    left join walmart.customer_information inf
    on o.customer_id = inf.customer_id
    left join walmart.customer_address ad
    on o.address_id = ad.address_id
    left join walmart.product p
    on t.product_id = p.product_id
    left join walmart.product_detailed de
    on p.sub_categories =  de.sub_categories
    order by t.transaction_id;
""")


In [94]:
# Get the result from SQL Script
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
df3 = pd.DataFrame(rows, columns=columns)
df3 = df3.sort_values(by='Row ID')
df3

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.00,41.91
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.00,219.58
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.62,2,0.00,6.87
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.58,5,0.45,-383.03
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.37,2,0.20,2.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,9990,CA-2014-110422,2014-01-21,2014-01-23,Second Class,TB-21400,Tom Boeckenhauer,Consumer,United States,Miami,...,33180,South,FUR-FU-10001889,Furniture,Furnishings,Ultra Door Pull Handle,25.25,3,0.20,4.10
9990,9991,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,92627,West,FUR-FU-10000747,Furniture,Furnishings,Tenex B1-RE Series Chair Mats for Low Pile Car...,91.96,2,0.00,15.63
9991,9992,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,92627,West,TEC-PH-10003645,Technology,Phones,Aastra 57i VoIP phone,258.58,2,0.20,19.39
9992,9993,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,92627,West,OFF-PA-10004041,Office Supplies,Paper,"It's Hot Message Books with Stickers, 2 3/4"" x 5""",29.60,4,0.00,13.32


In [95]:
# Get the columns from SQL Scripts
df3.columns

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit'],
      dtype='object')

In [97]:
df3.reset_index(drop=True, inplace=True)
df3

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.00,41.91
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.00,219.58
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.62,2,0.00,6.87
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.58,5,0.45,-383.03
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.37,2,0.20,2.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,9990,CA-2014-110422,2014-01-21,2014-01-23,Second Class,TB-21400,Tom Boeckenhauer,Consumer,United States,Miami,...,33180,South,FUR-FU-10001889,Furniture,Furnishings,Ultra Door Pull Handle,25.25,3,0.20,4.10
9990,9991,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,92627,West,FUR-FU-10000747,Furniture,Furnishings,Tenex B1-RE Series Chair Mats for Low Pile Car...,91.96,2,0.00,15.63
9991,9992,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,92627,West,TEC-PH-10003645,Technology,Phones,Aastra 57i VoIP phone,258.58,2,0.20,19.39
9992,9993,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,92627,West,OFF-PA-10004041,Office Supplies,Paper,"It's Hot Message Books with Stickers, 2 3/4"" x 5""",29.60,4,0.00,13.32


In [99]:
# Caomparing the columns headers to repare for checking
print(df.columns == df3.columns)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True]


In [100]:
# Chewck if normalization of data is correct
check = df.compare(df3)

In [101]:
# Check the differences
check

Sales         Discount        Profit         
        self   other     self other    self    other
0     261.96  261.96      NaN   NaN   41.91    41.91
1     731.94  731.94      NaN   NaN  219.58   219.58
2      14.62   14.62      NaN   NaN    6.87     6.87
3     957.58  957.58     0.45  0.45 -383.03  -383.03
4      22.37   22.37     0.20  0.20    2.52     2.52
...      ...     ...      ...   ...     ...      ...
9989     NaN     NaN     0.20  0.20    4.10     4.10
9990   91.96   91.96      NaN   NaN   15.63    15.63
9991  258.58  258.58     0.20  0.20   19.39    19.39
9992   29.60   29.60      NaN   NaN   13.32    13.32
9993  243.16  243.16      NaN   NaN   72.95    72.95

[9965 rows x 6 columns]